## Classification multi-tâches

On va expérimenter un problème de classification multi-tâches sur un jeu de données d'images, issu du travail suivant: 

3D Object Representations for Fine-Grained Categorization
       Jonathan Krause, Michael Stark, Jia Deng, Li Fei-Fei
       4th IEEE Workshop on 3D Representation and Recognition, at ICCV 2013 (3dRR-13). Sydney, Australia. Dec. 8, 2013.
       
Dans ce corpus, on a des images de voiture de différents modèles, de différentes marques et de différentes époques. 

On va essayer de prédire chacune de ses caractéristiques à partir de l'image. 

       

Il faut d'abord récupérer et désarchiver les fichiers suivants: 
    
   http://ai.stanford.edu/~jkrause/car196/car_ims.tgz
   
   http://ai.stanford.edu/~jkrause/car196/cars_annos.mat
   
   Si vous avez une connexion trop lente en salle, vous pouvez utiliser cet échantillon plus petit: 
   
   https://www.irit.fr/~Philippe.Muller/sample_cars.zip (10% du jeu de test complet)
   
Et mettez les dans un répertoire (par exemple Car_dataset)

In [1]:
!unzip http://ai.stanford.edu/~jkrause/car196/car_ims.tgz

'unzip' n'est pas reconnu en tant que commande interne
ou externe, un programme ex�cutable ou un fichier de commandes.


In [2]:
CAR_PATH = "./Car_dataset/"

In [3]:
import scipy.io
import numpy as np
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
import torchvision
from torchvision import datasets, transforms, models
import torchvision.models as models
from torchvision.transforms import ToTensor
from tqdm import tqdm
from glob import glob
from os.path import isfile, join
import os
from PIL import ImageFile
from PIL import Image
ImageFile.LOAD_TRUNCATED_IMAGES = True
from matplotlib import pyplot as plt

/home/muller/anaconda3/envs/visual/lib/python3.8/site-packages/torchvision/io/image.py:11: UserWarning: Failed to load image Python extension: /media/muller/disque 2/anaconda3/envs/visual/lib/python3.8/site-packages/torchvision/image.so: undefined symbol: _ZNK3c1010TensorImpl36is_contiguous_nondefault_policy_implENS_12MemoryFormatE
  warn(f"Failed to load image Python extension: {e}")


In [4]:
from cars import load_annotations, brand_dict, vehicle_types_dict

In [5]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [6]:
# pour l'entrainement en torch, on a besoin d'un générateur d'instances, 
# qui sera utilisé par le générateur de batch
class CarDataset(Dataset):
    def __init__(self,car_path,transform,translation_dict):
        self.path = car_path
        self.folder = [x for x in os.listdir(car_path)]
        self.transform = transform
        self.translation_dict = translation_dict

    def __len__(self):
        return len(self.folder)

    def __getitem__(self,idx):
        img_loc = os.path.join(self.path, self.folder[idx])
        image = Image.open(img_loc).convert('RGB')
        single_img = self.transform(image)

        label1 = translation_dict[self.folder[idx]][0]
        label2 = translation_dict[self.folder[idx]][1]
        label3 = translation_dict[self.folder[idx]][2]

        sample = {'image':single_img, 'labels': {'label_brand':label1, 
                                                 'label_vehicle_type':label2, 
                                                 'label_epoch':label3}}
        return sample   

In [29]:
translation_dict = load_annotations(CAR_PATH+"/cars_annos.mat")

### A faire

   - visualiez la distribution des classes de chaque tâche
   - est-ce qu'elles sont équilibrées ? 

In [8]:
data_transforms = transforms.Compose([
        transforms.Resize((224,224)),
        transforms.ToTensor(),
        transforms.Normalize((0.5,0.5,0.5), (0.5,0.5,0.5))
    ])

cardata = CarDataset(CAR_PATH+"/car_ims", transform=data_transforms,translation_dict=translation_dict)

# on garde seulement 20% du dataset pour le train, car il est gros
# pas la peine si vous avez pris l'échantillon faites 80%/20%/0
train_len = int(cardata.__len__()*0.2)
test_len  = int(cardata.__len__()*0.2)
ignored   = int(cardata.__len__()*0.6)
train_set, val_set, ignored_data = torch.utils.data.random_split(cardata, [train_len, test_len,ignored])

# DataLoader va générer les batchs
train_loader = DataLoader(train_set, batch_size=16, shuffle=True, 
                                num_workers=4, drop_last=True)
test_loader = DataLoader(val_set, batch_size=16, shuffle=False, 
                               num_workers=4, drop_last=True)

In [30]:
print(cardata[0]['image'].shape)

torch.Size([3, 224, 224])


### A faire

In [19]:
# A COMPLETER : il faut un modèle spécialisant chaque tâche
# essayez par exemple avec une couche linéaire simple
# ou une couche linéaire après une couche de drop-out
class MultilabelClassifier(nn.Module):
    def __init__(self, n_brand, n_vehicle_type, n_epoch):
        super().__init__()
        # la partie commune du modèle = resnet moins sa couche finale, dimension de sortie = 512
        self.resnet = models.resnet34(pretrained=True)
        self.model_wo_fc = nn.Sequential(*(list(self.resnet.children())[:-1]))

        self.brand = ...
        
        self.vehicle_type = ...
            
        self_epoch = ...

    def forward(self, x):
        x = self.model_wo_fc(x)
        x = torch.flatten(x, 1)

        
        return {
            'brand': 0,
            'vehicle_type': 0,
            'epoch': 0
        }

In [20]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = MultilabelClassifier(6,5,2).to(device)
print(device)

cuda


In [ ]:
# On définit ici la fonction de loss globale
# A FAIRE: quel autre choix pourrait-on faire ? 
#  si vous avez le temps essayez de changer cette loss globale
def criterion(loss_func,outputs,pictures):
    losses = 0
    for i, key in enumerate(outputs):
        losses += loss_func(outputs[key], pictures['labels'][f'label_{key}'].to(device))
    return losses

In [31]:
def training(model,device,lr_rate,epochs,train_loader):
    num_epochs = epochs
    losses = []
    checkpoint_losses = []

    optimizer = torch.optim.Adam(model.parameters(), lr=lr_rate)
    n_total_steps = len(train_loader)

    loss_func = nn.CrossEntropyLoss()

    for epoch in range(num_epochs):
        i = 0
        for pictures in tqdm(train_loader,desc=f"Epoque {epoch+1}"):
            images = pictures['image'].to(device)
            pictures = pictures

            outputs = model(images)

            loss = criterion(loss_func,outputs, pictures)
            losses.append(loss.item())

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            if (i+1) % (int(n_total_steps/1)) == 0:
                checkpoint_loss = torch.tensor(losses).mean().item()
                checkpoint_losses.append(checkpoint_loss)
                print (f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{n_total_steps}], Loss: {checkpoint_loss:.4f}')
            i = i + 1
    return checkpoint_losses

In [ ]:
## A FAIRE  entrainez le modèle

# checkpoint_losses =

In [36]:
# Fonction de calcul des performances en validation
# Que veut dire l'accuracy par classe ici ? corriger l'affichage
def validation(model, dataloader, *args):
    all_predictions = torch.tensor([]).to(device)
    all_true_labels = torch.tensor([]).to(device)

    with torch.no_grad():
            n_correct = []
            n_class_correct = []
            n_class_samples = []
            n_samples = 0

            for arg in args:
                n_correct.append(len(arg))
                n_class_correct.append([0 for i in range(len(arg))])
                n_class_samples.append([0 for i in range(len(arg))])

            # l'eval se fait par batch aussi, et sur le gpu si possible
            for pictures in tqdm(dataloader):
                images = pictures['image'].to(device)
                outputs = model(images)
                labels = [pictures['labels'][picture].to(device) for picture in pictures['labels']]

                for i,out in enumerate(outputs):
                    # quelle est la forme de outputs[out] ?
                    _, predicted = torch.max(outputs[out],axis=1)
                    n_correct[i] += (predicted == labels[i]).sum().item()

                    if i == 0:
                          n_samples += labels[i].size(0)

                    for k in range(16):
                        label = labels[i][k]
                        pred = predicted[k]
                        if (label == pred):
                            n_class_correct[i][label] += 1
                        n_class_samples[i][label] += 1

    return n_correct,n_samples,n_class_correct,n_class_samples

def class_acc(n_correct,n_samples,n_class_correct,n_class_samples,class_list):
    for i in range(len(class_list)):
          print("-------------------------------------------------")
          acc = 100.0 * n_correct[i] / n_samples
          print(f'Overall class performance: {round(acc,1)} %')
          for k in range(len(class_list[i])):
              acc = 100.0 * n_class_correct[i][k] / n_class_samples[i][k]
              print(f'Accuracy of {class_list[i][k]}: {round(acc,1)} %')
    print("-------------------------------------------------")



In [ ]:
classes_brand = list(brand_dict.values())
classes_vehicle_type = list(vehicle_types_dict.values())
classes_epoch = ['2009 and earlier','2010 and later']
class_list = [classes_brand,classes_vehicle_type,classes_epoch]

n_correct,n_samples,n_class_correct,n_class_samples = validation(model,test_loader,
 

In [35]:
class_acc(n_correct,n_samples,n_class_correct,n_class_samples,class_list)

In [ ]:
## A FAIRE : calculez les précision/rappel pour chaque classe